In [3]:
import torch
import os
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments)
from trl import SFTTrainer
from huggingface_hub import notebook_login

In [4]:
notebook_login()

In [5]:
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
        "tiiuae/falcon-7b-instruct",
        quantization_config=bnb_config,
        device_map={"": 0},
        #trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

c:\Users\BMSCE CSE\Desktop\Instruct\myenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# def print_trainable_parameters(model):
#     """
#     Prints the number of trainable parameters in the model.
#     """
#     trainable_params = 0
#     all_param = 0
#     for _, param in model.named_parameters():
#         all_param += param.numel()
#         if param.requires_grad:
#             trainable_params += param.numel()
#     print(
#         f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
#     )


In [7]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value"
        # "dense",
        # "dense_h_to_4h",
        # "dense_4h_to_h",
    ],
)

model.config.use_cache = False
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 18,874,368 || all params: 6,940,595,072 || trainable%: 0.2719416390698783


In [8]:
prompt = f"""
<human>: What does a cluster definition file contain??
<assistant>:
""".strip()
print(prompt)


<human>: What does a cluster definition file contain??
<assistant>:


In [9]:

training_arguments = TrainingArguments(
    output_dir="./InstructWithOurOwnData",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim='paged_adamw_32bit',
    save_steps=250,
    fp16=True,
    logging_steps=10,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    max_steps=320,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)

In [6]:
model.config.use_cache = False
import json
from datasets import DatasetDict, Dataset

# Load the JSON file
with open(r"C:\Users\BMSCE CSE\Downloads\HPEquestionsOurOwn.json", 'r') as json_file:
    data = json.load(json_file)

# Extract the "questions" data
questions_data = data["text"]

# Create a new list to store the reformatted data
formatted_data = []

# Iterate through each item in the original data and reformat it
for item in questions_data:
    # Check if both question and answer are strings before concatenating
    if isinstance(item["question"], str) and isinstance(item["answer"], str):
        formatted_item = {
            "text": "### Human: " + item["question"] + " " + "###Assistant: " +item["answer"]
        }
        formatted_data.append(formatted_item)
    else:
        print("Skipping item with non-string question or answer:", item)


formatted_dataset = Dataset.from_dict(data)

# Print the Dataset summary
dataset = formatted_dataset
print(dataset)
print(dataset['text'][0])

Dataset({
    features: ['text'],
    num_rows: 311
})
{'answer': 'Management switches on cluster systems are paired into switch stacks, with two switches per stack. The top switch is typically the master switch, while the bottom switch is typically the slave switch.', 'question': 'How are management switches typically paired in cluster systems, and what roles do the top and bottom switches in a stack usually play?'}


In [11]:
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=512,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     # packing=True,
#     packing=False,
# )
# Assuming 'dataset' is your dataset
dataset = dataset.map(lambda x: {'text': str(x['text'])})

# Now, you can initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)


Map:   0%|          | 0/311 [00:00<?, ? examples/s]

Map:   0%|          | 0/311 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [12]:

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

trainer.train()

#model.save_pretrained("output_dir")
trainer.push_to_hub()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aditi-cs21 (bmsce). Use `wandb login --relogin` to force relogin


  0%|          | 0/320 [00:00<?, ?it/s]

c:\Users\BMSCE CSE\Desktop\Instruct\myenv\Lib\site-packages\transformers\models\falcon\modeling_falcon.py:446: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = F.scaled_dot_product_attention(


{'loss': 2.8029, 'grad_norm': 0.45438313484191895, 'learning_rate': 0.0002, 'epoch': 0.13}
{'loss': 2.5882, 'grad_norm': 0.6797813773155212, 'learning_rate': 0.0002, 'epoch': 0.26}
{'loss': 2.3855, 'grad_norm': 0.6741534471511841, 'learning_rate': 0.0002, 'epoch': 0.39}
{'loss': 2.1299, 'grad_norm': 1.0471246242523193, 'learning_rate': 0.0002, 'epoch': 0.51}
{'loss': 2.1667, 'grad_norm': 0.9779266119003296, 'learning_rate': 0.0002, 'epoch': 0.64}
{'loss': 2.1876, 'grad_norm': 0.8094273209571838, 'learning_rate': 0.0002, 'epoch': 0.77}
{'loss': 2.0298, 'grad_norm': 0.9309846758842468, 'learning_rate': 0.0002, 'epoch': 0.9}
{'loss': 2.0028, 'grad_norm': 0.7519174814224243, 'learning_rate': 0.0002, 'epoch': 1.03}
{'loss': 1.9234, 'grad_norm': 0.8262293934822083, 'learning_rate': 0.0002, 'epoch': 1.16}
{'loss': 1.9232, 'grad_norm': 0.9563876986503601, 'learning_rate': 0.0002, 'epoch': 1.29}
{'loss': 1.8096, 'grad_norm': 1.4543246030807495, 'learning_rate': 0.0002, 'epoch': 1.41}
{'loss': 1

c:\Users\BMSCE CSE\Desktop\Instruct\myenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

c:\Users\BMSCE CSE\Desktop\Instruct\myenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Aditi25/InstructWithOurOwnData/commit/16dac3e4134e4fc84abf539fa18b31edae3992ae', commit_message='End of training', commit_description='', oid='16dac3e4134e4fc84abf539fa18b31edae3992ae', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
PEFT_MODEL = "Aditi25/InstructWithOurOwnData"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)


adapter_config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

c:\Users\BMSCE CSE\Desktop\Instruct\myenv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\BMSCE CSE\.cache\huggingface\hub\models--Aditi25--InstructWithOurOwnData. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

In [14]:
%%time
device = "cuda:0"
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

def generate_response(question: str) -> str:
    prompt = f"""
<human>: {question}
<assistant>:
""".strip()
    # encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

CPU times: total: 0 ns
Wall time: 0 ns


In [15]:
prompt = "How are management switches typically paired in cluster systems, and what roles do the top and bottom switches in a stack usually play?"
print(generate_response(prompt))

c:\Users\BMSCE CSE\Desktop\Instruct\myenv\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\BMSCE CSE\Desktop\Instruct\myenv\Lib\site-packages\transformers\generation\configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Management switches are typically paired in a cluster to ensure proper communication and fault tolerance. The top switch usually serves as the primary switch and is responsible for distributing network traffic, while the bottom switch usually serves as a backup switch and is responsible for distributing power to the network equipment.
User


In [17]:

prompt = "What procedure should users follow to configure management switches with a cluster definition file?"
print(generate_response(prompt))

Users should follow the procedure outlined in the cluster configuration guide to configure management switches with a cluster definition file. This involves creating a cluster definition file, specifying the switch model and cluster name, and then running the "cm init-cluster" command to create the cluster.
User


In [16]:
# import torch

# def query_model(model, tokenizer, input_text):
#     # Tokenize input text
#     input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

#     # Generate response from the model
#     with torch.no_grad():
#         output = model.generate(
#             input_ids,
#             do_sample=True,
#             max_length=50,
#             pad_token_id=tokenizer.eos_token_id,
#             num_return_sequences=1,
#         )
    
#     # Decode the generated response
#     response = tokenizer.decode(output[0], skip_special_tokens=True)

#     return response

# # Example usage
# user_input = "How are management switches typically paired in cluster systems, and what roles do the top and bottom switches in a stack usually play?"
# response = query_model(model, tokenizer, user_input)
# print(response)
